In [131]:
# set up autoreload
%reset
%load_ext autoreload
%autoreload 2

Nothing done.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [132]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from dataset import DataManager, FrameDataGenerator

# check gpu
print(tf.sysconfig.get_build_info())
print(tf.config.list_physical_devices('GPU'))


OrderedDict([('cpu_compiler', '/dt9/usr/bin/gcc'), ('cuda_compute_capabilities', ['sm_35', 'sm_50', 'sm_60', 'sm_70', 'sm_75', 'compute_80']), ('cuda_version', '11.8'), ('cudnn_version', '8'), ('is_cuda_build', True), ('is_rocm_build', False), ('is_tensorrt_build', True)])
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [133]:
# set up data generator
data_manager = DataManager(
        data_path=r"/home/ji/Dropbox/Robotics/ENPM809K_Fundamentals_in_AI_and_DL/Data",
        train_val_test_split=(0.80,0.1,0.1))

data_manager_small = DataManager(
        data_path=r"/home/ji/Dropbox/Robotics/ENPM809K_Fundamentals_in_AI_and_DL/Data_Test",
        train_val_test_split=(1.0,0.0,0.0))

2023-11-25 22:49:50.579 | INFO     | dataset:__init__:29 - Found video file: /home/ji/Dropbox/Robotics/ENPM809K_Fundamentals_in_AI_and_DL/Data/Data_2023-11-06_09-22-28.avi
2023-11-25 22:49:50.580 | INFO     | dataset:__init__:29 - Found video file: /home/ji/Dropbox/Robotics/ENPM809K_Fundamentals_in_AI_and_DL/Data/Data_2023-11-12_09-53-23.avi
2023-11-25 22:49:50.581 | INFO     | dataset:__init__:29 - Found video file: /home/ji/Dropbox/Robotics/ENPM809K_Fundamentals_in_AI_and_DL/Data/Data_2023-11-06_09-42-29.avi
2023-11-25 22:49:50.582 | INFO     | dataset:__init__:29 - Found video file: /home/ji/Dropbox/Robotics/ENPM809K_Fundamentals_in_AI_and_DL/Data/Data_2023-10-27_09-55-04.avi
2023-11-25 22:49:50.583 | INFO     | dataset:__init__:29 - Found video file: /home/ji/Dropbox/Robotics/ENPM809K_Fundamentals_in_AI_and_DL/Data/Data_2023-11-09_10-13-05.avi
2023-11-25 22:49:50.584 | INFO     | dataset:__init__:29 - Found video file: /home/ji/Dropbox/Robotics/ENPM809K_Fundamentals_in_AI_and_DL/Da

In [134]:
# set up tf database
config = {}
config["fold_n_frames"] = 6 # fold n frames to predict the n+1 frame
img_h, img_w = 192,256
config['frame_resize_reshape'] = (img_h,img_w)
# train/val/test data loader
train_loader = FrameDataGenerator(file_list=data_manager.get_training_files(),
                                    config=config)
val_loader = FrameDataGenerator(file_list=data_manager.get_validation_files(),
                                    config=config)
test_loader = FrameDataGenerator(file_list=data_manager.get_test_files(),
                                    config=config)
train_loader_small = FrameDataGenerator(file_list=data_manager_small.get_training_files(),
                                    config=config)

# create tensorflow database from the generator
n_col = 7 # the txt files have 7 columns
output_signature = ((tf.TensorSpec(shape = (img_h, img_w, 3*config["fold_n_frames"]),
                                    dtype = train_loader.frame_dtype),
                     tf.TensorSpec(shape = (n_col*config["fold_n_frames"],),
                                    dtype = train_loader.action_dtype)),
                    tf.TensorSpec(shape = (img_h, img_w, 3),
                                    dtype = train_loader.frame_dtype),
                    )
    
# create train/val/test tf dataset
train_ds = tf.data.Dataset.from_generator(train_loader,
                                          output_signature = output_signature)
val_ds = tf.data.Dataset.from_generator(val_loader,
                                          output_signature = output_signature)
test_ds = tf.data.Dataset.from_generator(test_loader,
                                          output_signature = output_signature)


# set up batch size
train_ds = train_ds.batch(8)
train_ds = train_ds.prefetch(buffer_size = 20)


# for validation and test evaluation, we are gonna sample
# mini batches and calculate the loss, and repeat several
# times and compute the mean loss, as opposed to compute 
# the loss on the whole validation/test set, so we can use
# the same data generation pipeline built for the training
# set
val_ds = val_ds.batch(8)
val_ds = val_ds.prefetch(buffer_size = 20)
test_ds = test_ds.batch(8)
test_ds = test_ds.prefetch(buffer_size = 20)


2023-11-25 22:49:50.911 | INFO     | dataset:__init__:127 - Found action file: /home/ji/Dropbox/Robotics/ENPM809K_Fundamentals_in_AI_and_DL/Data/Data_2023-11-06_09-22-28_merge.txt
2023-11-25 22:49:50.912 | INFO     | dataset:__init__:127 - Found action file: /home/ji/Dropbox/Robotics/ENPM809K_Fundamentals_in_AI_and_DL/Data/Data_2023-11-13_09-22-16_merge.txt
2023-11-25 22:49:50.912 | INFO     | dataset:__init__:127 - Found action file: /home/ji/Dropbox/Robotics/ENPM809K_Fundamentals_in_AI_and_DL/Data/Data_2023-11-06_09-27-29_merge.txt
2023-11-25 22:49:50.913 | INFO     | dataset:__init__:127 - Found action file: /home/ji/Dropbox/Robotics/ENPM809K_Fundamentals_in_AI_and_DL/Data/Data_2023-10-25_09-58-22_merge.txt
2023-11-25 22:49:50.914 | INFO     | dataset:__init__:127 - Found action file: /home/ji/Dropbox/Robotics/ENPM809K_Fundamentals_in_AI_and_DL/Data/Data_2023-10-26_10-30-08_merge.txt
2023-11-25 22:49:50.914 | INFO     | dataset:__init__:127 - Found action file: /home/ji/Dropbox/Robo

In [135]:
train_ds_small = tf.data.Dataset.from_generator(train_loader_small,
                                          output_signature = output_signature)
train_ds_small = train_ds_small.batch(8)
train_ds_small = train_ds_small.prefetch(5)

some functions for figuring out conv and conv2dtr dimensions

In [136]:
# image in
def outdim(d,f,p,s):
    return np.floor(d+2*p-f)/s+1

# image out size
def outdim_conv2d_tr(d,f,p,s, output_padding):
    return ((d-1)*s+f-2*p+output_padding)

d1 = outdim(192,8,0,2)
d2 = outdim(d1,6,0,2)
d3 = outdim(d2,6,0,2)
d4 = outdim(d3,4,0,2)
print(d4)

d4_ = outdim_conv2d_tr(d4,4,0,2,0)
d3_ = outdim_conv2d_tr(d4_,6,0,2,0)
d2_ = outdim_conv2d_tr(d3_,6,0,2,1)
d1_ = outdim_conv2d_tr(d2_,8,0,2,0)
print(d1_)

9.0
192.0


In [137]:
from model import EncoderNet, DecoderNet, InteractionModule, FramePredictionModel

# some hyper parameters for the network
encoder_output_dim = 1024


img_dim = (img_h,img_w,3*config["fold_n_frames"])

encoder = EncoderNet(layer_specs = [
                        {"type":"input","kwargs":{"input_shape":img_dim}},
                        {"type":"conv2d","kwargs":{"filters": 64,  "kernel_size": 8, "strides":2,"activation":"relu"}},
                        {"type":"conv2d","kwargs":{"filters": 128, "kernel_size": 6, "strides":2,"activation":"relu"}},
                        {"type":"conv2d","kwargs":{"filters": 128, "kernel_size": 6, "strides":2,"activation":"relu"}},
                        {"type":"conv2d","kwargs":{"filters": 128, "kernel_size": 4, "strides":2,"activation":"relu"}},
                        {"type":"flatten"},
                        {"type":"dense","kwargs":{"units":encoder_output_dim,"activation":"relu"}}
                        ]
                    )
# figure out output shape
encoder.compute_output_shape(img_dim)

# get the output size of the last conv layer
conv_out_size = encoder.output_shapes[3][1:]

# create the interaction module
interaction_dim, interaction_output_dim = 2048, 2048
interaction = InteractionModule(encoder_dim=encoder_output_dim,
                                action_dim=7,
                                intermediate_dim=interaction_dim,
                                output_dim=interaction_output_dim)

# fc_dim is the dimension of the first fc layer in decoder
nchan = 128
fc_dim = int(np.prod(conv_out_size[:2])*nchan)

decoder = DecoderNet(layer_specs = [
                        {"type":"input_shape","kwargs":{"shape":(interaction_output_dim,)}},
                        {"type":"dense","kwargs":{"units":fc_dim,"activation":"relu"}},
                        {"type":"reshape","kwargs":{"target_shape":(conv_out_size[0],conv_out_size[1],nchan)}},
                        {"type":"conv2dtr","kwargs":{"filters": 128, "kernel_size": 4, "strides":2,"activation":"relu"}},
                        {"type":"conv2dtr","kwargs":{"filters": 128, "kernel_size": 6, "strides":2,"activation":"relu"}},
                        {"type":"conv2dtr","kwargs":{"filters": 128, "kernel_size": 6, "strides":2,"activation":"relu", "output_padding":1}},
                        {"type":"conv2dtr","kwargs":{"filters":   3, "kernel_size": 8, "strides":2,"activation":None}},
                        ]
                    )

outshape_dec = decoder.compute_output_shape((None, interaction_output_dim))[1:]

# make sure decoder output matches the input image's size
assert(outshape_dec[:2]==list(img_dim[:2]))
assert(outshape_dec[-1]==3)


In [140]:
# create the full model
fpm = FramePredictionModel(encoder=encoder,
                           decoder=decoder,
                           interaction=interaction)

# test with some sample data
(frame_input_sample,actions_sample), frame_output_sample = next(iter(train_ds))
out_test = fpm((frame_input_sample, actions_sample))
assert(out_test.shape==frame_output_sample.shape)

# compile model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)#,clipvalue=0.5)
fpm.compile(optimizer=optimizer,
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tfa.metrics.r_square.RSquare(name='r2')]
            )

# Do an overfit on a small dataset

In [141]:
test_epochs = 10
history = fpm.fit(train_ds_small,epochs=test_epochs, verbose=1,steps_per_epoch=112)

Epoch 1/10
112/112 [==============================] - 45s 361ms/step - loss: 0.3241 - r2: -36.8244
Epoch 2/10
112/112 [==============================] - 41s 369ms/step - loss: 0.0895 - r2: -39.3791
Epoch 3/10
112/112 [==============================] - 42s 372ms/step - loss: 0.0932 - r2: -47.1656
Epoch 4/10
  9/112 [=>............................] - ETA: 39s - loss: 0.0458 - r2: -24.0511

KeyboardInterrupt: 

In [80]:
optimizer.learning_rate = 1e-5

In [81]:
history = fpm.fit(train_ds_small,epochs=test_epochs, verbose=1,steps_per_epoch=112)

Epoch 1/10
112/112 [==============================] - 39s 348ms/step - loss: 0.0181 - r2: -0.0409
Epoch 2/10
112/112 [==============================] - 39s 348ms/step - loss: 0.0178 - r2: -0.0338
Epoch 3/10
112/112 [==============================] - 39s 349ms/step - loss: 0.0177 - r2: -0.0329
Epoch 4/10
112/112 [==============================] - 39s 349ms/step - loss: 0.0178 - r2: -0.0317
Epoch 5/10
112/112 [==============================] - 39s 349ms/step - loss: 0.0178 - r2: -0.0316
Epoch 6/10
112/112 [==============================] - 40s 354ms/step - loss: 0.0177 - r2: -0.0317
Epoch 7/10
112/112 [==============================] - 39s 352ms/step - loss: 0.0178 - r2: -0.0320
Epoch 8/10
112/112 [==============================] - 38s 340ms/step - loss: 0.0178 - r2: -0.0294
Epoch 9/10
112/112 [==============================] - 38s 336ms/step - loss: 0.0177 - r2: -0.0303
Epoch 10/10
112/112 [==============================] - 39s 346ms/step - loss: 0.0177 - r2: -0.0314


In [86]:
input, output = next(iter(train_ds_small))

ex1 = fpm(input)

In [98]:
input[0].numpy()[0,:,:,:].squeeze().shape

(192, 256, 12)

In [ ]:
input

In [107]:
input[1][0]

<tf.Tensor: shape=(28,), dtype=float32, numpy=
array([-0.1360946 ,  2.6358507 , -0.13310656,  0.66286063,  1.0972971 ,
        0.09600986,  0.5013749 , -0.1360946 ,  2.6358507 , -0.13310656,
        0.66286063,  1.0972971 ,  0.09600986,  0.34872684, -0.1360946 ,
        2.6358507 , -0.13310656,  0.66286063,  1.0972971 ,  0.09600986,
       -0.24518125, -0.1360946 ,  2.6358507 , -0.13310656,  0.66286063,
        1.0972971 ,  0.09600986,  0.04735185], dtype=float32)>

In [108]:
input[1][1]

<tf.Tensor: shape=(28,), dtype=float32, numpy=
array([-0.1360946 ,  2.6358507 , -0.13310656,  0.66286063,  1.0972971 ,
        0.09600986, -0.03645438, -0.1360946 ,  2.6358507 , -0.13310656,
        0.66286063,  1.0972971 ,  0.09600986, -0.11709915, -0.1360946 ,
        2.6358507 , -0.13310656,  0.66286063,  1.0972971 ,  0.09600986,
       -0.14059892, -0.1360946 ,  2.6358507 , -0.13310656,  0.66286063,
        1.0972971 ,  0.09600986,  0.07185117], dtype=float32)>

In [109]:
input[1][2]

<tf.Tensor: shape=(28,), dtype=float32, numpy=
array([-0.1360946 ,  2.6358507 , -0.13310656,  0.66286063,  1.0972971 ,
        0.09600986,  0.08903697, -0.1360946 ,  2.6358507 , -0.13310656,
        0.66286063,  1.0972971 ,  0.09600986,  0.11516085, -0.1360946 ,
        2.6358507 , -0.13310656,  0.66286063,  1.0972971 ,  0.09600986,
       -0.01960179, -0.1360946 ,  2.6358507 , -0.13310656,  0.66286063,
        1.0972971 ,  0.09600986, -0.02587992], dtype=float32)>

In [142]:
# for plotting predicted frame
from PIL import Image
import time
import psutil

for _ in range(3):
    input, output = next(iter(train_ds_small))
    ex1 = fpm(input)
    data = ex1.numpy()[0,:,:,:3].squeeze()
    data = (data*255).astype(np.uint8)
    im = Image.fromarray(data)
    im.show()
    time.sleep(0.5)
    for proc in psutil.process_iter():
        if proc.name() == "display":
            proc.kill()

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/ji/snap/code/common/.cache/gio-modules/libgiolibproxy.so
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/ji/snap/code/common/.cache/gio-modules/libgiolibproxy.so
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/ji/snap/code/common/.cache/gio-modules/libgiolibproxy.so


In [90]:
im.close()